# Lab 4: Dependency parsing

In this assignment, you will implement a simplified version of the dependency parser presented by [Glavaš and Vulić (2021)](http://dx.doi.org/10.18653/v1/2021.eacl-main.270). This parser consists of a transformer encoder followed by a bi-affine layer that computes arc scores for all pairs of words. These scores are then used as logits in a classifier that predicts the position of the head of each word. In contrast to the parser described in the paper, yours will only support unlabelled parsing, that is, you will not implement what the authors call a *relation classifier*. As the encoder, you will use the [uncased DistilBERT base model](https://huggingface.co/docs/transformers/model_doc/distilbert) from the [Transformers](https://huggingface.co/docs/transformers/main/en/index) library, even though every other BERT-based encoder should work equally well.

*Tasks you can choose for the oral exam are marked with the graduation cap 🎓 emoji.*

## 🎓 Task 1: Read the paper

Start by reading the relevant parts of [Glavaš and Vulić (2021)](http://dx.doi.org/10.18653/v1/2021.eacl-main.270). Focus on Section&nbsp;3, which explains the overall structure of the dependency parser and details the role of the transformer encoder and bi-affine scoring. While reading, take notes on how the arc scores are computed and how they are used to determine dependency relations. These concepts will be critical for implementing the parser in later tasks.

In [1]:
#pip install conllu

## Dataset

The data for this lab comes from the English Web Treebank from the [Universal Dependencies Project](http://universaldependencies.org). To read the data, we use the [CoNLL-U Parser](https://pypi.org/project/conllu/) library. The code in the next cell defines a PyTorch [Dataset](https://pytorch.org/docs/stable/data.html#torch.utils.data.Dataset) wrapper for the data.

In [2]:
import conllu
from torch.utils.data import Dataset


class ParserDataset(Dataset):
    def __init__(self, filename):
        super().__init__()
        self.items = []
        with open(filename, "rt", encoding="utf-8") as fp:
            for tokens in conllu.parse_incr(fp):
                self.items.append(
                    [(t["form"], t["head"]) for t in tokens if isinstance(t["id"], int)]
                )

    def __len__(self):
        return len(self.items)

    def __getitem__(self, idx):
        return self.items[idx]

We can now load the training data:

In [3]:
TRAIN_DATA = ParserDataset("en_ewt-ud-train.conllu")

Our data consists of **parsed sentences**. A parsed sentence is represented as a list of pairs. The first component of each pair (a string) represents a word; the second component (an integer) specifies the position of the word’s head, i.e., its parent in the dependency tree. Note that word positions are numbered starting at&nbsp;1; the head position&nbsp;0 represents the root of the tree.

Run the next cell to see an example sentence:

In [4]:
EXAMPLE_WORDS, EXAMPLE_HEADS = zip(*TRAIN_DATA[531])

EXAMPLE_WORDS, EXAMPLE_HEADS

(('I', 'like', 'yuor', 'blog', '.'), (2, 0, 4, 2, 2))

The example sentence consists of five whitespace-separated words, including the final punctuation mark. The head of the pronoun *I* is the word at position&nbsp;2 – the verb *like*. The head of the word *like* is the root of the tree (position&nbsp;0). The dependents of *like* are *I* (position&nbsp;1), the noun *blog* (position&nbsp;4), and the final punctuation mark. Note that the pronoun *your* (position&nbsp;3) is misspelled as *yuor*.

## 🎓 Task 2: Tokenisation

To feed parsed sentences to DistilBERT, we need to tokenise them and encode the resulting tokens as integers in the model vocabulary. We start by loading the DistilBERT tokeniser using the [Auto classes](https://huggingface.co/docs/transformers/v4.37.2/en/model_doc/auto):

In [5]:
from transformers import AutoTokenizer

TOKENIZER = AutoTokenizer.from_pretrained("distilbert-base-uncased")

We can call the tokeniser on the example sentence as shown in the next cell. Note that we use the *is_split_into_words* keyword argument to indicate that the input is already pre-tokenised (split on whitespace).

In [6]:
EXAMPLE_TOKENS = TOKENIZER(EXAMPLE_WORDS, is_split_into_words=True)

The output of the tokeniser is an object of class [`BatchEncoding`](https://huggingface.co/docs/transformers/main_classes/tokenizer#transformers.BatchEncoding). The code in the following cell shows the list of tokens:

In [7]:
[TOKENIZER.decode(i) for i in EXAMPLE_TOKENS.input_ids]

['[CLS]', 'i', 'like', 'yu', '##or', 'blog', '.', '[SEP]']

As you can see, the tokeniser adds the special tokens `[CLS]` and `[SEP]` and splits unknown words (here: the misspelled word *yuor*) into sub-word tokens. We will need to keep track of which tokens correspond to which word. To achieve that, we can use the method [`word_to_tokens()`](https://huggingface.co/docs/transformers/main_classes/tokenizer#transformers.BatchEncoding.word_to_tokens), which gets us the encoded token span (an object of class [`TokenSpan`](https://huggingface.co/docs/transformers/internal/tokenization_utils#transformers.TokenSpan)) corresponding to a word in a sequence of the input batch:

In [8]:
for i, word in enumerate(EXAMPLE_WORDS):
    print(EXAMPLE_TOKENS.word_to_tokens(i), "->", word)

TokenSpan(start=1, end=2) -> I
TokenSpan(start=2, end=3) -> like
TokenSpan(start=3, end=5) -> yuor
TokenSpan(start=5, end=6) -> blog
TokenSpan(start=6, end=7) -> .


Your first task is to code a function `encode()` that takes a tokeniser and a list of sentences and returns the tokeniser&rsquo;s encoded input as well as the corresponding token spans. The following cell contains skeleton code for this function.

In [9]:
def encode(tokenizer, sentences):
    # TODO: Replace the next line with your own code
    raise NotImplementedError

Implement this function to match the following specification:

**encode** (*tokenizer*, *sentences*):

> Uses the specified *tokenizer* to encode a batch of parsed sentences (*sentences*). Returns a pair consisting of a [`BatchEncoding`](https://huggingface.co/docs/transformers/main_classes/tokenizer#transformers.BatchEncoding) and a matching batch of token spans (as explained above). The [`BatchEncoding`](https://huggingface.co/docs/transformers/main_classes/tokenizer#transformers.BatchEncoding) is the standard batch encoding, but with tensors instead of lists of Python integers. Sentences have been truncated to the maximum acceptable input length.

**Hint:** Read the documentation of [`PreTrainedTokenizer`](https://huggingface.co/docs/transformers/en/main_classes/tokenizer#transformers.PreTrainedTokenizer.__call__) to find out how to call the tokeniser.

In [10]:

def encode(tokenizer, sentences):
    # Extract only the words from each parsed sentence.
    words = [[word for word, _ in sentence] for sentence in sentences]

    # Encode using the tokenizer.
    encoded = tokenizer(
        words,
        is_split_into_words=True,
        return_tensors="pt",
        padding=True,
        truncation=True
    )

    # Build token spans: for each sentence and each word, get the corresponding TokenSpan.
    batch_token_spans = []
    for i, sentence in enumerate(words):
        spans = []
        for j in range(len(sentence)):
            # Note: tokenizer.word_to_tokens(i, j) returns the span for the j-th word in the i-th sentence.
            token_span = encoded.word_to_tokens(i, j)
            spans.append(token_span)
        batch_token_spans.append(spans)

    return encoded, batch_token_spans


### 🤞 Test your code

To test you code, call `encode()` on a small number of sentences and check that output matches your expectations.

In [11]:

# Define a small number of test sentences (as parsed word lists)
test_sentences = [
    [("The", "DET"), ("cat", "NOUN"), ("sat", "VERB"), ("on", "ADP"), ("the", "DET"), ("mat", "NOUN"), (".", "PUNCT")],
    [("She", "PRON"), ("enjoys", "VERB"), ("reading", "VERB"), ("books", "NOUN"), (".", "PUNCT")],
    [("Deep", "ADJ"), ("learning", "NOUN"), ("is", "AUX"), ("fascinating", "ADJ"), ("!", "PUNCT")]
]

# Call encode function
encoded_output, token_spans = encode(TOKENIZER, test_sentences)

# Print results
print("Encoded Input IDs:")
print(encoded_output["input_ids"])

print("\nToken Spans:")
for i, spans in enumerate(token_spans):
    print(f"Sentence {i+1}:")
    for j, span in enumerate(spans):
        print(f"  Word {j}: {span}")


Encoded Input IDs:
tensor([[  101,  1996,  4937,  2938,  2006,  1996, 13523,  1012,   102],
        [  101,  2016, 15646,  3752,  2808,  1012,   102,     0,     0],
        [  101,  2784,  4083,  2003, 17160,   999,   102,     0,     0]])

Token Spans:
Sentence 1:
  Word 0: TokenSpan(start=1, end=2)
  Word 1: TokenSpan(start=2, end=3)
  Word 2: TokenSpan(start=3, end=4)
  Word 3: TokenSpan(start=4, end=5)
  Word 4: TokenSpan(start=5, end=6)
  Word 5: TokenSpan(start=6, end=7)
  Word 6: TokenSpan(start=7, end=8)
Sentence 2:
  Word 0: TokenSpan(start=1, end=2)
  Word 1: TokenSpan(start=2, end=3)
  Word 2: TokenSpan(start=3, end=4)
  Word 3: TokenSpan(start=4, end=5)
  Word 4: TokenSpan(start=5, end=6)
Sentence 3:
  Word 0: TokenSpan(start=1, end=2)
  Word 1: TokenSpan(start=2, end=3)
  Word 2: TokenSpan(start=3, end=4)
  Word 3: TokenSpan(start=4, end=5)
  Word 4: TokenSpan(start=5, end=6)


## 🎓 Task 3: Merging tokens

DistilBERT gives us a representation for each *token* in a sentence. To compute scores between pairs of *words*, we will need to combine the token representations that correspond to each word. A standard strategy for this is to take their element-wise mean. The next cell contains skeleton code for a function `merge_tokens()` that implements this strategy.

In [12]:
def merge_tokens(tokens, token_spans):
    # TODO: Replace the next line with your own code
    raise NotImplementedError

Implement this function to match the following specification:

**merge_tokens** (*tokens*, *token_spans*)

> Takes a batch of token vectors (*tokens*) and a list of matching token spans (*token_spans*) and returns a batch of word-level representations, computed using the element-wise mean. The token vectors are a tensor of shape (*batch_size*, *num_tokens*, *hidden_dim*), where *hidden_dim* is the dimensionality of the DistilBERT representations. The token spans are a nested list containing integer pairs, as computed in Task&nbsp;1. The result is a tensor of shape (*batch_size*, *max_num_words*, *hidden_dim*), where *max_num_words* denotes the maximum number of words in any sentence in the batch. Entries corresponding to padding are represented by the zero vector of size *hidden_dim*.

In [13]:

def merge_tokens(tokens, token_spans):
    batch_size, _, hidden_dim = tokens.shape
    max_num_words = max(len(spans) for spans in token_spans)
    merged = torch.zeros(batch_size, max_num_words, hidden_dim, device=tokens.device)
    for b in range(batch_size):
        spans = token_spans[b]
        for w_idx, (start, end) in enumerate(spans):
            if start >= tokens.size(1) or end > tokens.size(1) or start >= end:
                continue
            merged[b, w_idx] = torch.mean(tokens[b, start:end, :], dim=0)
    return merged

### 🤞 Test your code

The code in the following cell creates a sample input to `merge_tokens()` and compares the output to a manually constructed control.

In [15]:
from transformers import TokenSpan
import torch
import torch.nn as nn


def test_merge_tokens():
    # Set the random seed for reproducibility
    torch.manual_seed(42)

    # Construct a test example
    subwords_test = torch.rand(1, 1 + len(EXAMPLE_WORDS) + 1, 3)

    # Add the expected token spans
    token_spans = [
        [TokenSpan(start=i, end=j) for i, j in [(1, 2), (2, 3), (3, 5), (5, 6), (6, 7)]]
    ]

    # Get the output of `merge_tokens()` and the expected output
    print("merged:", merge_tokens(subwords_test, token_spans))
    print("reference:", torch.mean(subwords_test[0, 3:5], dim=0))

test_merge_tokens()

merged: tensor([[[0.9593, 0.3904, 0.6009],
         [0.2566, 0.7936, 0.9408],
         [0.5013, 0.7512, 0.6673],
         [0.4294, 0.8854, 0.5739],
         [0.2666, 0.6274, 0.2696]]])
reference: tensor([0.5013, 0.7512, 0.6673])


###### Feedback:The instruction mentions: “The result is a tensor of shape (batch_size, max_num_words, hidden_dim), where max_num_words denotes the maximum number of words in any sentence in the batch. Entries corresponding to padding are represented by the zero vector of size hidden_dim.” Your code is not returning a tensor. For this it will be important that you add the padding tokens (e.g. zero vectors), so that the shape matches for each item in the batch. 
###### solution:correction has been made

## 🎓 Task 4: Biaffine layer

Your next task is to implement the bi-affine layer. Given matrices $X \in \mathbb{R}^{m \times d}$ and $X' \in \mathbb{R}^{n \times d}$, this layer computes a matrix $Y \in \mathbb{R}^{m \times n}$ as

$$
Y = X W X'{}^\top + b
$$

where $W \in \mathbb{R}^{d \times d}$ and $b \in \mathbb{R}$ are learnable weight and bias parameters. In the context of the dependency parser, the matrices $X$ and $X'$ hold the word representations of all dependents and all heads in the input sentence, and the entries of the matrix $Y$ are interpreted as scores of possible dependency arcs. More specifically, the entry $Y_{ij}$ represents the score of an arc from a head word at position&nbsp;$j$ to a dependent at position&nbsp;$i$.

The following cell contains skeleton code for the implementation of the bi-affine layer. Implement this layer according to the specification above. Initialise the weights as in [`nn.Linear`](https://github.com/pytorch/pytorch/blob/v2.6.0/torch/nn/modules/linear.py#L50).

In [16]:
import torch.nn as nn


class Biaffine(nn.Module):
    def __init__(self, encoder_dim):
        super().__init__()
        # TODO: Replace the next line with your own code
        raise NotImplementedError

    def forward(self, x1, x2):
        # TODO: Replace the next line with your own code
        raise NotImplementedError

In [17]:

class Biaffine(nn.Module):
    def __init__(self, encoder_dim):
        super().__init__()
        self.weight = nn.Parameter(torch.empty(encoder_dim, encoder_dim))
        self.bias = nn.Parameter(torch.empty(1))
        nn.init.kaiming_uniform_(self.weight, a=5**0.5)
        self.bias.data.zero_()

    def forward(self, x1, x2):
        x1W = torch.matmul(x1, self.weight)
        scores = torch.bmm(x1W, x2.transpose(1, 2))
        scores += self.bias
        return scores

**⚠️ Note that your implementation should be able to handle *batches* of input sentences.**

### 🤞 Test your code

The test creates a sample input to the bi-affine layer as well as suitable weights and biases and checks that the output of the `forward()` method matches a manually constructed reference output.

In [18]:
def test_biaffine():
    import torch.testing

    # Test values
    batch_size = 2
    x1_size = 3
    x2_size = 5
    encoder_dim = 7

    # Create two random inputs
    x1 = torch.rand(batch_size, x1_size, encoder_dim)
    x2 = torch.rand(batch_size, x2_size, encoder_dim)

    # Create the biaffine layer
    m = Biaffine(encoder_dim)

    # Compute a reference output using for loops
    reference = torch.zeros(batch_size, x1_size, x2_size)
    for b in range(batch_size):
        for x1i in range(x1_size):
            for x2i in range(x2_size):
                tmp = 0
                for i in range(encoder_dim):
                    for j in range(encoder_dim):
                        tmp += x1[b,x1i,i] * m.weight[i,j] * x2[b,x2i,j]
                reference[b,x1i,x2i] = tmp + m.bias

    # Check that the two are identical
    torch.testing.assert_close(m.forward(x1, x2), reference)

## 🎓 Task 5: Parser

We are now ready to put the two main components of the parser together: the encoder (DistilBert) and the bi-affine layer that computes the arc scores. We also add a dropout layer between the two components.

The following code cell contains skeleton code for the parsing model with the `init()` method already complete. Your task is to implement the `forward()` method. If you are unsure how things should be wired up, have another look at the slides.

In [19]:
import torch.nn as nn
from transformers import DistilBertModel, DistilBertPreTrainedModel


class DistilBertForParsing(DistilBertPreTrainedModel):
    def __init__(self, config, dropout=0.1):
        super().__init__(config)
        self.config = config
        self.distilbert = DistilBertModel(config)
        self.dropout = nn.Dropout(dropout)
        self.biaffine = Biaffine(config.hidden_size)

    def forward(self, encoded_input, token_spans):
        # TODO: Replace the next line with your own code
        raise NotImplementedError

Implement the `forward()` method to match the following specification. Annotate all intermediate tensors with their shapes.

**forward** (*encoded_input*, *token_spans*)

> Takes a tokeniser-encoded batch of sentences (*encoded_input*, of type `BatchEncoding`) and a corresponding nested list of token spans (*token_spans*) and returns a tensor with scores for each pair of words. More specifically, the output tensor $Y$ has shape (*batch_size*, *num_words*, *num_words+1*), where the entry $Y_{bij}$ represents the score of an arc from a head word at position&nbsp;$j$ to a dependent at position&nbsp;$i$ in the $b$th sentence of the batch. Note that the number of possible heads is one greater than the number of possible dependents because the possible heads include the root of the dependency tree, which we represent using the special token `[CLS]` (at position&nbsp;0).

In [20]:


class DistilBertForParsing(DistilBertPreTrainedModel):
    def __init__(self, config, dropout=0.1):
        super().__init__(config)
        self.distilbert = DistilBertModel(config)
        self.dropout = nn.Dropout(dropout)
        self.biaffine = Biaffine(config.hidden_size)

    def forward(self, encoded_input, token_spans):
        outputs = self.distilbert(**encoded_input)
        last_hidden_state = outputs.last_hidden_state

        #Merge token representations into word-level representations.
        merged_words = merge_tokens(last_hidden_state, token_spans)

        # Include the [CLS] token as a candidate head (for the root).
        # Assume [CLS] is at position 0 in the token_reps.
        cls_embeds = last_hidden_state[:, 0, :].unsqueeze(1)

        # Concatenate cls_reps with word representations for heads.
        head_reprs = torch.cat([cls_embeds, merged_words], dim=1)

        # Apply dropout on the word representations (for dependents).
        merged_words = self.dropout(merged_words)

        head_reprs = self.dropout(head_reprs)
        # Compute the bi-affine scores.
        arc_scores = self.biaffine(merged_words, head_reprs)

        return arc_scores

###### Feedback:Why are you not using the merge_tokens function here? It seems that something is wrong with the encoded input, and it looks like it is a list instead of a tensor. The issue might be that you do not set “return_tensors="pt"” when calling the tokenizer. 
###### solution: correction made

### 🤞 Test your code

The testing code instantiates the parsing model and feeds it a small batch of sentences.

In [21]:
def test_model():
    m = DistilBertForParsing.from_pretrained("distilbert-base-uncased")
    encoded_input, token_spans = encode(TOKENIZER, TRAIN_DATA[:2])
    return m(encoded_input, token_spans)

## Data loader

We are now almost ready to train the parser. The missing piece is a data collator that prepares a batch of parsed sentences:

* tokenises the sentences and extracts token spans using `encode()` (Task&nbsp;1)
* constructs the ground-truth head tensor needed to compute the loss (Task&nbsp;2)

The code in the next cell implements these two steps. For pseudo-words introduced through padding, we assign a head position of −100. This value is ignored by PyTorch’s cross-entropy loss function.

In [22]:
import torch


class ParserBatcher(object):
    def __init__(self, tokenizer, device=None):
        self.tokenizer = tokenizer
        self.device = device

    def __call__(self, parser_inputs):
        encoded_input, start_indices = encode(self.tokenizer, parser_inputs)

        # Get the maximal number of words, for padding
        max_num_words = max(len(s) for s in parser_inputs)

        # Construct tensor containing the ground-truth heads
        all_heads = []
        for parser_input in parser_inputs:
            words, heads = zip(*parser_input)
            heads = list(heads)
            heads.extend([-100] * (max_num_words - len(heads)))  # -100 will be ignored
            all_heads.append(heads)
        all_heads = torch.LongTensor(all_heads)

        # Send all data to the specified device
        if self.device:
            encoded_input = encoded_input.to(self.device)
            all_heads = all_heads.to(self.device)

        return encoded_input, start_indices, all_heads

## Training loop

Finally, here is the training loop of the parser. Most of it is quite standard. The training loss of the parser is the cross-entropy between the head scores and the ground truth head positions. In other words, the parser is trained as a classifier that predicts the position of each word&rsquo;s head.

In [23]:
import torch.nn.functional as F
from torch.optim import Adam
from torch.utils.data import DataLoader
from tqdm import tqdm

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")


def train(dataset, n_epochs=1, lr=1e-5, batch_size=8):
    # Initialise the tokeniser
    tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

    # Initialise the encoder
    model = DistilBertForParsing.from_pretrained("distilbert-base-uncased").to(DEVICE)

    # Initialise the data loader
    data_loader = DataLoader(
        dataset,
        batch_size=batch_size,
        collate_fn=ParserBatcher(tokenizer, device=DEVICE),
    )

    # Initialise the optimiser
    optimizer = Adam(model.parameters(), lr=lr)

    # Train for the specified number of epochs
    for epoch in range(n_epochs):
        model.train()

        # We keep track of the running loss
        running_loss = 0
        n_batches = 0
        with tqdm(total=len(dataset)) as pbar:
            pbar.set_description(f"Epoch {epoch + 1}")

            # Process a batch of samples
            for encoded_input, token_spans, gold_heads in data_loader:
                optimizer.zero_grad()

                # Compute the arc scores
                arc_scores = model.forward(encoded_input, token_spans)
                # shape: [batch_size, num_words, num_words+1]

                # Flatten arc_scores and gold_heads for cross_entropy
                loss = F.cross_entropy(arc_scores.flatten(0, -2), gold_heads.view(-1))
                # shape of the flattened arc_scores: [batch_size * num_words, num_words+1]
                # shape of the flattened gold_heads: [batch_size * num_words]

                # Backward pass
                loss.backward()
                optimizer.step()

                # Update the loss
                running_loss += loss.item()
                n_batches += 1
                pbar.set_postfix(loss=running_loss / n_batches)
                pbar.update(len(token_spans))

    return model

We are now ready to train the parser. With a GPU, you should expect training times of approximately 3&nbsp;minutes per epoch.

In [24]:
PARSING_MODEL = train(TRAIN_DATA, n_epochs=1)

Some weights of DistilBertForParsing were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['biaffine.bias', 'biaffine.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Epoch 1: 100%|████████████████████████████████████| 12544/12544 [47:15<00:00,  4.42it/s, loss=0.734]


## 🎓 Task 6: Evaluation

Dependency parsers are commonly evaluated using **unlabelled attachment score (UAS)**, which is the percentage of (non-root) words that have been assigned their correct heads. The following cell contains skeleton code for a function `uas()` that computes this score on a given dataset.

In [25]:
def uas(tokenizer, model, filename, batch_size=8):
    # TODO: Replace the following line with your own code
    return 0.0

Implement the `uas()` function to match the following specification:

**uas** (*tokenizer*, *model*, *filename*)

> Takes a tokenizer (*tokenizer*), a trained parsing model (*model*), and the filename of a dataset in the CoNLLU format (*filename*) and returns the unlabelled attachment score of the model on the tokenised dataset.

In [26]:

def uas(tokenizer, model, filename, batch_size=8):
    dataset = ParserDataset(filename)
    data_loader = DataLoader(
        dataset,
        batch_size=batch_size,
        collate_fn=ParserBatcher(tokenizer, device=DEVICE),
        shuffle=False
    )
    model.eval()
    total = 0
    correct = 0
    with torch.no_grad():
        for encoded_input, _, gold_heads in data_loader:
            arc_scores = model(encoded_input, _)
            predicted_heads = arc_scores.argmax(dim=-1)
            mask = gold_heads != -100
            correct += ((predicted_heads == gold_heads) & mask).sum().item()
            total += mask.sum().item()
    return correct / total if total else 0.0

**⚠️ Note that pseudo-words corresponding to padding must be excluded from the calculation of the UAS.**

The code in the following cell evaluates the trained parser on the development section of the data:

In [27]:
uas(TOKENIZER, PARSING_MODEL, "en_ewt-ud-dev.conllu")

0.8818243270110143

**Your notebook must contain output demonstrating at least 86% UAS on the development data.**

**🥳 Congratulations on finishing this lab! 🥳**